In [1]:
import json
import pandas as pd
import os
from typing import List
from datetime import datetime

In [2]:
ML_JOB_PATH = "machine-learning-jobs20221109161820.jsonl"
DS_JOB_PATH = "sample20221109145023.jsonl"

In [3]:
def validate_data_path(data_path:str):
    """Validates data path

    Args:
        data_path (str): path of the data

    Raises:
        FileNotFoundError: File does not exist
    """
    if not os.path.exists(data_path):
        raise FileNotFoundError(data_path)

In [4]:
# not all data point is perfect, for those with errors I will ignore

def return_data(data_path:str)->List:
    """Returns job detail from on job posting

    Args:
        data_path (str): Path of the data

    Returns:
        List: A list full of the job posting, each a dictionary
    """
    validate_data_path(data_path)
    data_list = []
    with open(data_path, 'r') as reader:
        for line in reader.readlines():
            data = json.loads(line)
            # Errors are only contained in the dictionary
            if isinstance(data,dict):
                continue
            data_list.append(data)
    return data_list

In [5]:
ML_JOB_RAW_DATA = return_data(ML_JOB_PATH)
DS_JOB_RAW_DATA = return_data(DS_JOB_PATH)

In [6]:
len(ML_JOB_RAW_DATA)

224

In [7]:
len(DS_JOB_RAW_DATA)

1212

In [8]:
TOTAL_JOB_RAW_DATA = ML_JOB_RAW_DATA + DS_JOB_RAW_DATA

In [9]:
len(TOTAL_JOB_RAW_DATA)

1436

In [10]:
# Attempting to fetch headers from their respective cols
headers = {}
for row in TOTAL_JOB_RAW_DATA:
    for col in range(1,len(row)):
        for key in row[col].keys():
            if str(key) + '_' + str(col) in headers:
                headers[f"{key}_{col}"] +=1
            else: 
                headers[f"{key}_{col}"] = 1

In [11]:
# These are all of the headers found
headers

{'company_name_1': 1436,
 'small_section_2': 1436,
 'Responsibilities:\xa0_3': 6,
 '\xa0_3': 10,
 'Essential Skills:_3': 2,
 'Career Level_4': 1436,
 'Qualification_4': 1434,
 'Years of Experience_4': 1241,
 'Job Type_4': 1436,
 'Job Specializations_4': 1436,
 'Registration No._4': 1429,
 'Company Size_4': 1411,
 'Industry_4': 1434,
 'Benefits & Others_4': 1151,
 'Company Overview_5': 1419,
 'url_6': 1436,
 'Average Processing Time_4': 1140,
 'NLP\xa0and\xa0Knowledge Graph Engineer, Sensors & Machine Learning Lab_3': 2,
 'Responsibilities:_3': 103,
 'Preferred Requirements:_3': 7,
 'About ASUS\xa0\xa0_3': 2,
 'About AICS\xa0_3': 2,
 'Job Responsibilities:_3': 42,
 'Requirements:\xa0_3': 9,
 'Specific Location_4': 63,
 'Solution Lead, Sensors & Machine Learning Lab_3': 2,
 'Description_3': 24,
 'Responsibilities_3': 240,
 'Requirements_3': 97,
 'Thank you for your interest\xa0and\xa0application to this role. Please note that only short-listed\xa0candidates\xa0will be contacted._3': 7,
 

In [12]:
# These headers have values for all jobs

full_header = [k for k,v in headers.items() if v == len(TOTAL_JOB_RAW_DATA)]
full_header

['company_name_1',
 'small_section_2',
 'Career Level_4',
 'Job Type_4',
 'Job Specializations_4',
 'url_6']

I am expecting all information to be filled in:
- First col: Name of the company
- Second col: Details of the job including pay, location (General and or specific), 
- Fourth col: Misc details of the company including size of company, dress code etc
- Fifth col: Company Overview, details on the company
- Sixth col: URL

Within the second col I expect there are also some headers to be further broken down. 

The largest one will have to be the second col with the job description being the most variable component of the posting. I suspect most of the work will be done in here to uncover as much information as possible.

The desired outcome will be a consolidated pandas dataframe with as little information loss as possible.

# Preprocess

## Full Columns

In [13]:
data = {}
job_title,career_level,job_type, job_specialisations, url = [],[],[],[],[]
for row in TOTAL_JOB_RAW_DATA:
    job_title.append(row[0]['position'])
    career_level.append(row[4]['Career Level'])
    job_type.append(row[4]['Job Type'])
    job_specialisations.append(row[4]['Job Specializations'])
    url.append(row[6]['url'])

data['job_title'] = job_title
data['career_level'] = career_level
data['job_type'] = job_type
data['job_specialisations'] = job_specialisations
data['url'] = url

assert (len(job_title) == len(career_level)==len(job_type)==len(job_specialisations)==len(url))

In [14]:
# Already we can see some field contains some other information like the job_title containing job type, location and company name for instance
pd.DataFrame.from_dict(data).tail()

,job_title,career_level,job_type,job_specialisations,url
1431,"Research Associate (Bioprinting), (R00009608) ...",Junior Executive,Full-Time,"Sciences, Science & Technology",http://www.jobstreet.com.sg/en/job/research-as...
1432,"Data Science Analyst, TikTok #Urgent #WorkNow*",Entry Level,Full-Time,"Computer/Information Technology, IT-Software",http://www.jobstreet.com.sg/en/job/data-scienc...
1433,"Data Analyst - Senior (1 year contract, Central)",Senior Executive,Temporary,"Manufacturing, Purchasing/Material Mgmt",http://www.jobstreet.com.sg/en/job/data-analys...
1434,Junior Machine Learning Engineer,Junior Executive,Full-Time,"Computer/Information Technology, IT-Software",http://www.jobstreet.com.sg/en/job/junior-mach...
1435,Senior Process Development Scientist (Organic ...,Senior Executive,Full-Time,"Sciences, Chemistry",http://www.jobstreet.com.sg/en/job/senior-proc...


## Partial Columns

### Small Section

This is where the trouble/fun starts. Lets start with the small section


In [15]:
# The small section contains a list with location, salary (if available) and then post date
for idx, row in enumerate(TOTAL_JOB_RAW_DATA):
    if idx > 20 :
        break
    else:
        print(row[2])

{'small_section': ['Central', 'SGD\xa06,000 - SGD\xa012,000', 'Posted on 4-Nov-22']}
{'small_section': ['Singapore', 'Posted 1 hour ago']}
{'small_section': ['Singapore', 'Posted on 8-Nov-22']}
{'small_section': ['Central - Others', 'Posted on 6-Nov-22']}
{'small_section': ['Singapore', 'Posted on 8-Nov-22']}
{'small_section': ['Singapore', 'Posted on 7-Nov-22']}
{'small_section': ['Novena', 'Posted on 7-Nov-22']}
{'small_section': ['Singapore', 'Posted on 7-Nov-22']}
{'small_section': ['Singapore', 'Posted on 5-Nov-22']}
{'small_section': ['Singapore', 'Posted on 7-Nov-22']}
{'small_section': ['Singapore', 'Posted on 5-Nov-22']}
{'small_section': ['Central - Others', 'SGD\xa04,000 - SGD\xa06,000', 'Posted on 3-Nov-22']}
{'small_section': ['Singapore', 'Posted on 7-Nov-22']}
{'small_section': ['West', 'Posted 11 hours ago']}
{'small_section': ['Singapore', 'Posted on 4-Nov-22']}
{'small_section': ['Singapore', 'Posted on 6-Nov-22']}
{'small_section': ['Singapore', 'Posted 11 hours ago'

In [16]:
demo = 'Posted on 7-Nov-22'

In [17]:
demo.split("Posted on")[1].strip()

'7-Nov-22'

In [18]:
datetime.strptime(demo.split("Posted on")[1].strip(),'%d-%b-%y')

datetime.datetime(2022, 11, 7, 0, 0)

In [19]:
curr_date = ML_JOB_PATH[21:-12]

In [20]:
curr_date

'20221109'

In [21]:
datetime.strptime(curr_date, '%Y%m%d').year

2022

In [22]:
def extract_date_from_posted_date(sample_text:str, file_path:str)->datetime:
    """Return posted date in datetime format

    Args:
        sample_text (str): sample text from the row
        file_path (str): File path from the data in jsonl format
        to determine the current date

    Returns:
        datetime: When the job was posted
    """
    if 'on' in sample_text:
        sample_date = sample_text.split("Posted on")[1].strip()
        return datetime.strptime(sample_date,'%d-%b-%y')
    else:
        curr_date = file_path[21:-12]
        return datetime.strptime(curr_date, '%Y%m%d')

In [45]:
# For location, we will take in anything that is other than Singapore, if Singapore we will leave it as unspecified
# For Salary, we will clean up the html artifact '\xa0' and seperate into low to high
# For Post date, we will just take in as is. Any variant of Posted on xxx will be tagged as the date of extraction which is collected on the jsonl file

location, low_salary, high_salary, post_date = [],[],[],[]

curr_date = ML_JOB_PATH[21:-12]

for idx, row in enumerate(TOTAL_JOB_RAW_DATA):
    select_row = row[2]['small_section']
    # location
    try:
        # There are instances where no location is entered 
        if select_row[0] is not None: 
            if 'sin' not in select_row[0].lower():
                location.append(select_row[0])
            else:
                location.append('unspecified')    
        else:
            location.append('unspecified')
        # check for salary, otherwise its posted date
        if 'sgd' in select_row[1].lower():
            if "\xa0" in select_row[1].lower():
                low_salary.append(select_row[1].split("\xa0")[1].split('-')[0].strip())
                high_salary.append(select_row[1].split("\xa0")[2].strip())
            else:
                salary = select_row[1].lower().split('sgd')[1].strip()
                low_salary.append(salary)
                high_salary.append(salary)
        # if sgd not in this value, it must be posted date
        else:
            salary = 'unspecified'
            low_salary.append(salary)
            high_salary.append(salary)
            post_date.append(extract_date_from_posted_date(select_row[1], ML_JOB_PATH))
        # if third value is present, it must be posted date
        if len(select_row) == 3:
            post_date.append(extract_date_from_posted_date(select_row[1], ML_JOB_PATH))
    except Exception as e:
        print(f"{idx} with {row} has issue {e}")

assert (len(location)==len(low_salary)==len(high_salary)==len(post_date))

In [47]:
data['location'] = location
data['low_salary'] = low_salary
data['high_salary'] = high_salary
data['post_date'] = post_date

### Job description

In [ ]:
jd_headers = [key for key in headers.keys() if key.endswith('3')]

In [ ]:
jd_headers

['Responsibilities:\xa0_3',
 '\xa0_3',
 'Essential Skills:_3',
 'NLP\xa0and\xa0Knowledge Graph Engineer, Sensors & Machine Learning Lab_3',
 'Responsibilities:_3',
 'Preferred Requirements:_3',
 'About ASUS\xa0\xa0_3',
 'About AICS\xa0_3',
 'Job Responsibilities:_3',
 'Requirements:\xa0_3',
 'Solution Lead, Sensors & Machine Learning Lab_3',
 'Description_3',
 'Responsibilities_3',
 'Requirements_3',
 'Thank you for your interest\xa0and\xa0application to this role. Please note that only short-listed\xa0candidates\xa0will be contacted._3',
 'Job Responsibilities_3',
 'Required Attributes_3',
 'Job Description_3',
 'Purpose of the post_3',
 'Qualifications_3',
 'More Information_3',
 'About the Client_3',
 'Main Duties & Responsibilities_3',
 'Experience and Qualifications_3',
 'Interest & Application_3',
 'Job Requirements_3',
 'For Research Fellow Position:_3',
 'Requirements:_3',
 'Next Step:_3',
 'expected package_3',
 'reasons for leaving_3',
 '._3',
 '[email\xa0protected]_3',
 'Rol